# 深度学习模型实现与测试

本笔记本包含多个卷积神经网络(CNN)深度学习模型的实现，包括 LeNet、AlexNet、VGG、NiN、GoogLeNet、ResNet 和 DenseNet。每个模型都可以单独测试，并使用 FashionMNIST 数据集进行训练和评估。

以下是主要内容：
- 使用 PyTorch 实现经典的深度学习模型。
- 提供模型训练和评估的通用函数。
- 测试不同模型的性能。

请根据需要选择模型并运行相应的代码块。

In [4]:
#导包
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [5]:
#通过Sequential类来实现LeNet模型，卷积神经网络LeNet（2个卷积层，3个全连接层）(使用批量归一化层)
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(1,6,5),
            #nn.BatchNorm2d(6),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6,16,5),
            #nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2)
        )
        self.fc=nn.Sequential(
            nn.Linear(16*4*4,120),
            #nn.BatchNorm1d(120),
            nn.Sigmoid(),
            nn.Linear(120,84),
            #nn.BatchNorm1d(84),
            nn.Sigmoid(),
            nn.Linear(84,10)
        )
    def forward(self,img):
        feature = self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output

#深度神经网络AlexNet(5个卷积层，3个全连接层)
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(1,96,11,4),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
            nn.Conv2d(96,256,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
            nn.Conv2d(256,384,3,1,1),
            nn.ReLU(),
            nn.Conv2d(384,384,3,1,1),
            nn.ReLU(),
            nn.Conv2d(384,256,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(3,2)
        )
        self.fc=nn.Sequential(
            nn.Linear(256*5*5,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,10)
        )
    def forward(self,img):
        feature=self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output

#使用重复元素的网络VGG-11(8个卷积层，三个全连接层)
class VGG(nn.Module):
    ratio=8
    conv_arch=((1,1,64//ratio),(1,64//ratio,128//ratio),(2,128//ratio,256//ratio),(2,256//ratio,512//ratio),(2,512//ratio,512//ratio))
    fc_features=512*7*7//ratio
    fc_hidden_units=4096//ratio
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.conv=nn.Sequential()
        for i,(num_convs,in_channels,out_channels) in enumerate(self.conv_arch):
            self.conv.add_module("vgg_block_"+str(i+1),self.vgg_block(num_convs,in_channels,out_channels))
        self.fc=nn.Sequential(
            nn.Linear(self.fc_features,self.fc_hidden_units),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(self.fc_hidden_units,self.fc_hidden_units),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(self.fc_hidden_units,10)
        )
    def forward(self,img):
        features=self.conv(img)
        output=self.fc(features.view(img.shape[0],-1))
        return output

    def vgg_block(self,num_convs, in_channels,out_channels):
        blk=[]
        for i in range(num_convs):
            if i==0:
                blk.append(nn.Conv2d(in_channels,out_channels,3,padding=1))
            else:
                blk.append(nn.Conv2d(out_channels,out_channels,3,padding=1))
            blk.append(nn.ReLU())
        blk.append(nn.MaxPool2d(2,2))
        return nn.Sequential(*blk)

#网络中的网络(NiN)(由卷积层和全连接层交替组成，其中全连接层用1*1卷积层来代替)
class NiN(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.net=nn.Sequential(
            self.nin_block(1,96,11,4,0),
            nn.MaxPool2d(3,2),
            self.nin_block(96,256,5,1,2),
            nn.MaxPool2d(3,2),
            self.nin_block(256,384,3,1,1),
            nn.MaxPool2d(3,2),
            nn.Dropout(0.5),
            self.nin_block(384,10,3,1,1),
        )
    def forward(self,img):
        tmp=self.net(img)
        output=F.avg_pool2d(tmp,tmp.shape[2:]).view(img.shape[0],-1)
        return output
    def nin_block(self,in_channels,out_channels,kernel_size,stride,padding):
        blk=nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding),
            nn.ReLU(),
            nn.Conv2d(out_channels,out_channels,1),
            nn.ReLU(),
            nn.Conv2d(out_channels,out_channels,1),
            nn.ReLU()
        )
        return blk

#含并行连结的网络（GoogLeNet)
class GoogLeNet(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        class Inception(nn.Module):
            def __init__(self, in_c,c1,c2,c3,c4):
                super().__init__()
                self.p1=nn.Sequential(
                    nn.Conv2d(in_c,c1,1),
                    nn.ReLU()
                )
                self.p2=nn.Sequential(
                    nn.Conv2d(in_c,c2[0],1),
                    nn.Conv2d(c2[0],c2[1],3,padding=1),
                    nn.ReLU()
                )
                self.p3=nn.Sequential(
                    nn.Conv2d(in_c,c3[0],1),
                    nn.Conv2d(c3[0],c3[1],5,padding=2),
                    nn.ReLU()
                )
                self.p4=nn.Sequential(
                    nn.MaxPool2d(3,stride=1,padding=1),
                    nn.Conv2d(in_c,c4,1),
                    nn.ReLU()
                )
            def forward(self,x):
                p1=self.p1(x)
                p2=self.p2(x)
                p3=self.p3(x)
                p4=self.p4(x)
                return torch.cat((p1,p2,p3,p4),dim=1)
        class GlobalAvgPool2d(nn.Module):
            def __init__(self, *args, **kwargs):
                super().__init__(*args, **kwargs)
            def forward(self,x):
                return F.avg_pool2d(x,x.shape[2:]).view(x.shape[0],-1)
        self.b1=nn.Sequential(
            nn.Conv2d(1,64,7,2,3),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1)
        )
        self.b2=nn.Sequential(
            nn.Conv2d(64,64,1),
            nn.Conv2d(64,192,3,1),
            nn.MaxPool2d(3,2,1)
        )
        self.b3=nn.Sequential(
            Inception(192,64,(96,128),(16,32),32),
            Inception(256,128,(128,192),(32,96),64),
            nn.MaxPool2d(3,2,1)
        )
        self.b4=nn.Sequential(
            Inception(480,192,(96,208),(16,48),64),
            Inception(512,160,(112,224),(24,64),64),
            Inception(512,128,(128,256),(24,64),64),
            Inception(512,112,(144,288),(32,64),64),
            Inception(528,256,(160,320),(32,128),128),
            nn.MaxPool2d(3,2,1)
        )
        self.b5=nn.Sequential(
            Inception(832,256,(160,320),(32,128),128),
            Inception(832,384,(192,384),(48,128),128),
            GlobalAvgPool2d()
        )
        self.net=nn.Sequential(
            self.b1,
            self.b2,
            self.b3,
            self.b4,
            self.b5,
            nn.Linear(1024,10)
        )
    def forward(self,img):
        return self.net(img)

#残差网络(ResNet-18)
class ResNet(nn.Module):
    class Residual(nn.Module):
            def __init__(self, in_channels,out_channels,stride=1):
                super().__init__()
                self.residual=nn.Sequential(
                    nn.Conv2d(in_channels,out_channels,3,stride,1),
                    nn.BatchNorm2d(out_channels),
                    nn.ReLU(),
                    nn.Conv2d(out_channels,out_channels,3,padding=1),
                    nn.BatchNorm2d(out_channels)
                )
                if in_channels!=out_channels:
                    self.change=nn.Conv2d(in_channels,out_channels,1,stride)
                else:
                    self.change=None
            def forward(self,x):
                y=self.residual(x)
                if self.change:
                    x=self.change(x)
                return F.relu(y+x)
    class GlobalAvgPool2d(nn.Module):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
        def forward(self,x):
            return F.avg_pool2d(x,x.shape[2:]).view(x.shape[0],-1)
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.net=nn.Sequential(
            nn.Conv2d(1,64,7,2,3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1),
            self.resnet_block(64,64,2,True),
            self.resnet_block(64,128,2),
            self.resnet_block(128,256,2),
            self.resnet_block(256,512,2),
            self.GlobalAvgPool2d(),
            nn.Linear(512,10)
        )
    def forward(self,img):
        return self.net(img)
    def resnet_block(self,in_channels,out_channels,num_residuals,first_block=False):
        blk=[]
        for i in range(num_residuals):
            if i==0 and not first_block:
                blk.append(self.Residual(in_channels,out_channels,stride=2))
            else:
                blk.append(self.Residual(out_channels,out_channels))
        return nn.Sequential(*blk)

#稠密连接网络(DenseNet)
class DenseNet(nn.Module):
    class DenseBlock(nn.Module):
        def __init__(self,num_convs,in_channels,out_channels):
            super().__init__()
            net=[]
            self.out_channels=in_channels
            for i in range(num_convs):
                net.append(self.conv_block(self.out_channels,out_channels))
                self.out_channels+=out_channels
            self.net=nn.ModuleList(net)
        def forward(self,X):
            for blk in self.net:
                Y=blk(X)
                X=torch.cat((X,Y),dim=1)
            return X
        def conv_block(self,in_channels,out_channels):
            blk=nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels,out_channels,3,padding=1)
            )
            return blk
    class GlobalAvgPool2d(nn.Module):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
        def forward(self,x):
            return F.avg_pool2d(x,x.shape[2:]).view(x.shape[0],-1)
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.net=nn.Sequential(
            nn.Conv2d(1,64,7,2,3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1),
        )
        num_channels,growth_rate=64,32
        num_convs_in_dense_blocks=[4,4,4,4]
        for i,num_convs in enumerate(num_convs_in_dense_blocks):
            DB=self.DenseBlock(num_convs,num_channels,growth_rate)
            self.net.add_module(f"DenseBlock_{i}",DB)
            num_channels=DB.out_channels
            if i!=len(num_convs_in_dense_blocks)-1:
                self.net.add_module(f"transition_block{i}",self.transition_block(num_channels,num_channels//2))
                num_channels=num_channels//2
        self.net.add_module("BN",nn.BatchNorm2d(num_channels))
        self.net.add_module("relu",nn.ReLU())
        self.net.add_module("global_avg_pool",self.GlobalAvgPool2d())
        self.net.add_module("fc",nn.Linear(num_channels,10))
    def forward(self,img):
        return self.net(img)
    def transition_block(self,in_channels,out_channels):
        blk=nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels,out_channels,1),
            nn.AvgPool2d(2,2)
        )
        return blk
    

In [6]:
#评估模型
def evaluate_accuracy(data_iter,net):
    acc_sum,n=0.0,0
    with torch.no_grad():
        for X,y in data_iter:
            acc_sum+=(net(X).argmax(dim=1)==y).float().sum().item()
            n+=y.shape[0]
    return acc_sum/n
#训练模型
def train_model(net,train_iter,loss,optimizer,num_epochs,device=None):
    print(f"training on {device}")
    for epoch in range(num_epochs):
        n,train_acc=0,0.0 
        for X,y in train_iter:
            X=X.to(device)
            y=y.to(device)
            net=net.to(device)
            y_hat=net(X)
            l=loss(y_hat,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_acc+=(y_hat.argmax(dim=1)==y).float().sum().item()
            l*=y.shape[0]
            n+=y.shape[0]
        print(f"epoch:{epoch},loss:{l/n},train_acc:{train_acc/n}")
    print("training end!")

In [7]:
'''
此区域用于测试模型
'''
#该段代码用于LeNet神经网络测试
"""
net=LeNet()
batch_size=256
#获取数据集和训练模型
mnist_train=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=True,download=True,transform=transforms.ToTensor())
mnist_test=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=False,download=True,transform=transforms.ToTensor())
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size,True)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size,False)
#定义设备，损失函数，优化器，训练次数
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=0.001)
num_epochs=20
train_model(net,train_iter,loss,optimizer,num_epochs,device=device)
print(evaluate_accuracy(test_iter,net.cpu()))
"""
#该段代码用于AlexNet深度神经网络测试
"""
net=AlexNet()
batch_size=256
#获取数据集和训练模型
#变换列表并组合
trans=[]
trans.append(torchvision.transforms.Resize(224))
trans.append(torchvision.transforms.ToTensor())
transform=torchvision.transforms.Compose(trans)
mnist_train=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=True,download=True,transform=transform)
mnist_test=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=False,download=True,transform=transform)
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size,True)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size,False)
#定义设备，损失函数，优化器，训练次数
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=0.001)
num_epochs=50
train_model(net,train_iter,loss,optimizer,num_epochs)
evaluate_accuracy(test_iter,net.cpu())
"""
#该段代码用于VGG-11(使用重复元素的神经网络)测试
"""
net=VGG()
print(net)
batch_size=256
#获取数据集和训练模型
#变换列表并组合
trans=[]
trans.append(torchvision.transforms.Resize(224))
trans.append(torchvision.transforms.ToTensor())
transform=torchvision.transforms.Compose(trans)
mnist_train=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=True,download=True,transform=transform)
mnist_test=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=False,download=True,transform=transform)
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size,True)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size,False)
#定义设备，损失函数，优化器，训练次数
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=0.001)
num_epochs=50
train_model(net,train_iter,loss,optimizer,num_epochs,device=device)
evaluate_accuracy(test_iter,net.cpu())
"""
#该段代码用于NiN测试
"""
net=NiN()
print(net)
batch_size=256
#获取数据集和训练模型
#变换列表并组合
trans=[]
trans.append(torchvision.transforms.Resize(224))
trans.append(torchvision.transforms.ToTensor())
transform=torchvision.transforms.Compose(trans)
mnist_train=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=True,download=True,transform=transform)
mnist_test=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=False,download=True,transform=transform)
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size,True)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size,False)
#定义设备，损失函数，优化器，训练次数
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=0.001)
num_epochs=50
train_model(net,train_iter,loss,optimizer,num_epochs,device=device)
evaluate_accuracy(test_iter,net.cpu())
"""
#该段代码用于GoogLeNet测试
"""
net=GoogLeNet()
print(net)
batch_size=256
#获取数据集和训练模型
#变换列表并组合
trans=[]
trans.append(torchvision.transforms.Resize(96))
trans.append(torchvision.transforms.ToTensor())
transform=torchvision.transforms.Compose(trans)
mnist_train=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=True,download=True,transform=transform)
mnist_test=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=False,download=True,transform=transform)
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size,True)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size,False)
#定义设备，损失函数，优化器，训练次数
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=0.001)
num_epochs=50
train_model(net,train_iter,loss,optimizer,num_epochs,device=device)
evaluate_accuracy(test_iter,net.cpu())
"""
#该段代码用于ResNet测试
"""
net=ResNet()
print(net)
batch_size=256
#获取数据集和训练模型
#变换列表并组合
trans=[]
trans.append(torchvision.transforms.Resize(96))
trans.append(torchvision.transforms.ToTensor())
transform=torchvision.transforms.Compose(trans)
mnist_train=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=True,download=True,transform=transform)
mnist_test=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=False,download=True,transform=transform)
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size,True)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size,False)
#定义设备，损失函数，优化器，训练次数
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=0.001)
num_epochs=50
train_model(net,train_iter,loss,optimizer,num_epochs,device=device)
evaluate_accuracy(test_iter,net.cpu())
"""
#该段代码用于DenseNet测试
net=DenseNet()
print(net)
batch_size=256
#获取数据集和训练模型
#变换列表并组合
trans=[]
trans.append(torchvision.transforms.Resize(96))
trans.append(torchvision.transforms.ToTensor())
transform=torchvision.transforms.Compose(trans)
mnist_train=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=True,download=True,transform=transform)
mnist_test=torchvision.datasets.FashionMNIST(root="./DataSets/FashionMNIST",train=False,download=True,transform=transform)
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size,True)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size,False)
#定义设备，损失函数，优化器，训练次数
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=0.001)
num_epochs=50
train_model(net,train_iter,loss,optimizer,num_epochs,device=device)
print(evaluate_accuracy(test_iter,net.cpu()))


DenseNet(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (DenseBlock_0): DenseBlock(
      (net): ModuleList(
        (0): Sequential(
          (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
          (2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (1): Sequential(
          (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
          (2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (2): Sequential(
          (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
          (2): Conv2d(128, 32, kernel_si

In [8]:
'''
此处用于做个人代码测试
'''

'\n此处用于做个人代码测试\n'